In [2]:
import numpy as np
import pandas as pd

import geopandas as gpd
import osmnx as ox

# Selecting New York City

First get city boundaries:

In [10]:
nyc_boundary = gpd.read_file('../d01_data/geo/nyc/ct-nyc-2020.geojson').dissolve()

Now get city ZCTAs (check both 2010 and 2020 files):

In [47]:
#National files:
ZCTA_2010 = gpd.read_file('../../../static/ZBP/spatial/tl_2020_us_zcta510.shp').to_crs(nyc_boundary.crs)
ZCTA_2020 = gpd.read_file('../../../static/ZBP/spatial/tl_2020_us_zcta520.shp').to_crs(nyc_boundary.crs)

#New York files:
nyc_2010 = ZCTA_2010.overlay(nyc_boundary)
nyc_2020 = ZCTA_2020.overlay(nyc_boundary)

#Collect:
valid_ZCTAs = {2010:nyc_2010.ZCTA5CE10.astype(int).values,
               2020:nyc_2020.ZCTA5CE20.astype(int).values}

## Querying ZBP

In [65]:
US_yearly_df  = {2000+y: pd.read_csv(f'../../../static/ZBP/raw/zbp{yy}detail.txt', usecols=['zip','naics','est'], encoding='latin')
                 for y in range(17,23)}
NYC_yearly_df = {year: df.loc[df['zip'].astype(int).isin(valid_ZCTAs[2020 if year>=2020 else 2010])].reset_index(drop=True)
                 for year,df in US_yearly_df.items()}
NYC_df_all = pd.concat([df.assign(year=year) for year, df in NYC_yearly_df.items()],ignore_index=True)
NYC_df_all.to_csv('../../../static/ZBP/all_NYC.csv', index=False)